In [ ]:
# Check device number
!ls -ltrh /dev/video*

In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=640, height=480, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

In [ ]:
# set your categories
CATEGORIES = ['car', 'mouse', 'remote']

# Set project directory
data = "custom"

In [ ]:
import cv2
import os
import datetime
import random
import ipywidgets
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

points = [{
    "cat": CATEGORIES[0],
    "top": True,
    "topX": -1,
    "topY": -1,
    "bottomX": -1,
    "bottomY": -1,
    "color": (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
}]
point = 0

Annotations = "./data/" + data + "/Annotations/"
ImageSets = "./data/" + data + "/ImageSets/Main/"
ImageSetsTrain = ImageSets + "train.txt"
ImageSetsTrainval = ImageSets + "trainval.txt"
ImageSetsTest = ImageSets + "test.txt"
ImageSetsVal = ImageSets + "val.txt"
JPEGImages = "./data/" + data + "/JPEGImages/"
labels = "./data/" + data + "/labels.txt"

# 检查并创建目录
directories = [Annotations, ImageSets, JPEGImages]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)


# snapshot_widget = ipywidgets.Image(value=default_image, width=camera.width, height=camera.height)
# traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
def darm(camera):
    global points, point
    snapshot = camera.copy()
    for i, p in enumerate(points):
        # print(p)
        if p['bottomX'] >= 0 & p['bottomY'] >= 0:
            # 创建一个和原图像同样大小的透明图像
            overlay = snapshot.copy()

            # 在透明图像上绘制矩形
            if point == i:
                cv2.rectangle(overlay, (p['topX'], p['topY']), (p['bottomX'], p['bottomY']), p['color'] + (255,), 4)
                alpha = 1  # 设置透明度
            else:
                cv2.rectangle(overlay, (p['topX'], p['topY']), (p['bottomX'], p['bottomY']), p['color'] + (100,), 4)
                alpha = 0.2  # 设置透明度

            # 使用alpha blending将透明图像和原图像混合在一起
            
            snapshot = cv2.addWeighted(overlay, alpha, snapshot, 1 - alpha, 0)
        else:
            snapshot = cv2.circle(snapshot, (p['topX'], p['topY']), 4, p['color'], 3)
    return bgr8_to_jpeg(snapshot)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=darm)



# update counts when we select a new category

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        if points[point]['top']:
            points[point]['top'] = False
            points[point]['topX'] = data['offsetX']
            points[point]['topY'] = data['offsetY']
            points[point]['top_x_widget'].value = points[point]['topX']
            points[point]['top_y_widget'].value = points[point]['topY']
            points[point]['bottomX'] = -1
            points[point]['bottomY'] = -1
            points[point]['bottom_x_widget'].value = points[point]['bottomX']
            points[point]['bottom_y_widget'].value = points[point]['bottomY']
            
        else:
            points[point]['top'] = True
            points[point]['bottomX'] = data['offsetX']
            points[point]['bottomY'] = data['offsetY']
            points[point]['bottom_x_widget'].value = points[point]['bottomX']
            points[point]['bottom_y_widget'].value = points[point]['bottomY']
            
        

        
camera_widget.on_msg(save_snapshot)


points_widget = ipywidgets.VBox([])

def add_ponit(p, index):
    # create widgets
    category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='category')

    top_x_widget = ipywidgets.IntText(description='Top X:')
    top_y_widget = ipywidgets.IntText(description='Top Y:')
    bottom_x_widget = ipywidgets.IntText(description='Bottom X:')
    bottom_y_widget = ipywidgets.IntText(description='Bottom Y:')

    def update_top_x_widget(change):
        # global topX
        p['topX'] = change['new']
    top_x_widget.observe(update_top_x_widget, names='value')

    def update_top_y_widget(change):
        # global topY
        p['topY'] = change['new']
    top_y_widget.observe(update_top_y_widget, names='value')

    def update_bottom_x_widget(change):
        # global bottomX
        p['bottomX'] = change['new']
    bottom_x_widget.observe(update_bottom_x_widget, names='value')

    def update_bottom_y_widget(change):
        # global bottomY
        p['bottomY'] = change['new']
    bottom_y_widget.observe(update_bottom_y_widget, names='value')
    
    choose_button = ipywidgets.Button(description='Choose')

    
    del_button = ipywidgets.Button(description='Del')

    
    box_layout = widgets.Layout(border='3px solid rgb{}'.format(p['color'][::-1]), margin_bottom='10px', background_color='lightblue')
    point_widget = ipywidgets.VBox([
        ipywidgets.HBox([category_widget, choose_button, del_button]),
        ipywidgets.HBox([top_x_widget, top_y_widget,]),
        ipywidgets.HBox([bottom_x_widget, bottom_y_widget]),
    ], layout=box_layout)
    p['category_widget'] = category_widget
    p['top_x_widget'] = top_x_widget
    p['top_y_widget'] = top_y_widget
    p['bottom_x_widget'] = bottom_x_widget
    p['bottom_y_widget'] = bottom_y_widget
    p['choose_button'] = choose_button
    p['point_widget'] = point_widget
    
    def choose(button):
        global point
        point = index
        for xi, xp in enumerate(points):
            if point == xi:
                choose_button.style.button_color = '#4898F8'
            else:
                xp['choose_button'].style.button_color = None
        
    choose_button.on_click(choose)
    
    def del_b(button):
        global point
        new_children = [child for child in points_widget.children if child != point_widget]
        points_widget.children = tuple(new_children)
        if len(points) - 1 == index:
            point = point - 1
        del points[index]

        points[point]['choose_button'].style.button_color = '#4898F8'

    del_button.on_click(del_b)
    points_widget.children = (*points_widget.children, point_widget)
    return p

for index, p in enumerate(points):
    add_ponit(p, index)
    if point == index:
        p['choose_button'].style.button_color = '#4898F8'
    else:
        p['choose_button'].style.button_color = None


# Add this to create a button for saving images
save_button = ipywidgets.Button(description='Save Image')

# Add this to define a function that will be called when the button is clicked
def save_image(b):
    global data, camera, points
    
    # 获取当前时间
    current_time = datetime.datetime.now().strftime("%Y%m%d")

    # 生成10位随机数
    random_number = random.randint(1000000000, 9999999999)

    # 构建文件名
    filename = f"{current_time}_{random_number}"
    
    cv2.imwrite(JPEGImages + filename+'.jpg', camera.value)
    
    # Save the bounding box coordinates and category to a text file
    with open(Annotations + filename + '.xml', 'a') as txtfile:  # Change 'data.txt' to your desired filename
        # Write the data for this image
        txtfile.write(f"""<annotation>
    <filename>{filename}</filename>
    <folder>{data}</folder>
    <source>
        <database>{data}</database>
        <annotation>{data}</annotation>
        <image>{data}</image>
    </source>
    <size>
        <width>{camera.width}</width>
        <height>{camera.height}</height>
        <depth>{camera.value.shape[2]}</depth>
    </size>
    <segmented>0</segmented>
""")
        for p in points:
            txtfile.write(f"""    <object>
        <name>{p['category_widget'].value}</name>
        <pose>unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{p['topX']}</xmin>
            <ymin>{p['topY']}</ymin>
            <xmax>{p['bottomX']}</xmax>
            <ymax>{p['bottomY']}</ymax>
        </bndbox>
    </object>
""")
        txtfile.write(f"""</annotation>""")  # Separate entries with a blank line
        # 打开文件以追加写入模式
    with open(ImageSetsTrain, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
    
    with open(ImageSetsTrainval, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")

    with open(ImageSetsTest, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
        
    with open(ImageSetsVal, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
    
    print('Image saved.')

# Add this to make the button call the function when clicked
save_button.on_click(save_image)



def add_point_new(button):
    
    global add_point, ponits
    
    p = add_ponit({
    "cat": CATEGORIES[0],
    "top": True,
    "topX": -1,
    "topY": -1,
    "bottomX": -1,
    "bottomY": -1,
    "color": (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
}, len(points))
    points.append(p)
    
    
    
add_point_button = ipywidgets.Button(description='Add New Point')
add_point_button.on_click(add_point_new)



data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, ]),
    add_point_button,
    points_widget,
    save_button
])

display(data_collection_widget)
print("data_collection_widget created")